In [ ]:
% Initialize COBRA
initCobraToolbox

In [ ]:
cd('/Users/pablodigiusto/Documents/GitHub/Whole-Cell-Network-Reconstruction-for-CHO-cells/Notebooks');
addpath('/Users/pablodigiusto/Documents/GitHub/Whole-Cell-Network-Reconstruction-for-CHO-cells/Notebooks/Matlab code');

In [ ]:
% Load iCHO3000 model
model = readCbModel('iCHOv3_CHO_28032024.mat');

In [ ]:
% Loading the RNA-seq data
dataTable = readtable('../Data/Zela Data/20200307_Bio141-148_merged.tpm.tsv', 'FileType', 'text', 'Delimiter', '\t');

% Extracting gene names if the first column
geneNames = dataTable{:, 1};

% Calculate mean expression across replicates for simplicity
expressionWT = mean(table2array(dataTable(:, 2:18)), 2); % Mean across WT replicates - THIS IS WRONG NEEDS TO BE FIXED
expressionZeLa = mean(table2array(dataTable(:, 19:end)), 2); % Mean across ZeLa replicates  - THIS IS WRONG NEEDS TO BE FIXED
expressionDataMatrix = [expressionWT, expressionZeLa]; % Combine into a single matrix

% Condition Names
cellNames = {'WT', 'ZeLa'}; % Adjust as needed

% Creating the expressionData Structure
expressionData = struct;
expressionData.gene = geneNames;
expressionData.valuebyTissue = expressionDataMatrix;
expressionData.Tissue = cellNames;

% Conditions:
% WT_P2 (S01,S07,S13)
% WT_P4 
% WT_P6
% WT_P8
% WT_P12
% WT_P14

% ZeLa_P2 (S22, S27, S33, S38)
% ZeLa_P4
% ZeLa_P6
% ZeLa_P8
% ZeLa_P12
% ZeLa_P14

In [ ]:
expressionWT

In [ ]:
% extract expression data of the genes in the model
modelData = getModelData(expressionData,model);

In [ ]:
modelData

In [ ]:
% Calculate enzymes in the model
spec = getSpecialistEnzymes(model);  
prom = getPromEnzymes(model);

In [ ]:
% Calculate enzyme expression
enzymeData = comparePromiscuousSpecific(spec,prom,modelData);

In [ ]:
edgeX = [-2 -1 0 1 2 2.5 3 4]; % bins  

In [ ]:
k = 3

In [ ]:
distMethod = 'euclidean'; % distance method  

In [ ]:
linkageMethod = 'complete'; % linkage metric for hierarchical clustering

In [ ]:
% Calculate clusters of enzyme expression
clustObj = geneExprDist_hierarchy(enzymeData,[],edgeX,k,distMethod,linkageMethod);